In [1]:
import os
import tempfile
import subprocess
from datetime import datetime

from concurrent.futures import ProcessPoolExecutor
from multiprocessing import Pool
from collections.abc import Mapping

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
sns.set(rc={'figure.figsize': (15, 10), 'figure.dpi' : 100})

import nevergrad as ng
from SALib.sample import saltelli
from SALib.analyze import sobol
import geotopy as gtp

/home/max/.local/share/virtualenvs/MHPC-project-WaUKRq_w/lib/python3.8/site-packages/geotopy/__init__.py:35: RuntimeWarning: Default GEOTop executable not found, check your installation
  warnings.warn("Default GEOTop executable not found, check your installation",


In [2]:
# Defaults

# System settings
systmpfs = '/tmp'

# Optimizer settings
num_workers = 2
budget = 16
algorithm = 'OnePlusOne'
timeout = 60

# High-Performance Derivative-Free Optimization for the GE🌍top Model Calibration

### Stefano Campanella

#### Giacomo Bertoldi, Alberto Sartori, Emanuele Cordano

## GEOtop Calibration

GEOtop is a physical model that simulates the heat and water budgets at and below the soil surface. 

Some inputs are unknown/uncertain: find the values such that the ouputs better reproduce the experimental data.

Case study: LTSER sites in Val di Mazia, Alto Adige

## Challenges

1. Many parameters 
2. No good prior
3. Time consuming simulations

<span class=fragment>**Parallel Derivative-free heuristic algorithms are the only option**</span>

Some points to keep in mind:

* Community of scientists, not HPC experts nor programmers
* Wide range of use cases and applications
* Calibration is CPU bounded
* For these algorithms, <span class="fragment highlight-red">scaling can be a tricky subject</span>

Consider genetic algorithms/PSO.

<span class=fragment>Increasing the number of phenotypes/particles increases the coverage of the parameters space in each generation/iteration, but the principle of operation of these algorithms lies in the correlation between one generation/iteration and the next.</span>

<span class=fragment>Hence, scaling must take into account that cutting the time to solution by increasing the number of processing units and decreasing the number of iterations can lead to worse results.</span>

## The Goal

1. Write a reusable calibration tool, with a simple yet general enough interface
2. Make it easily deployable on HPC systems
3. Perform the optimization and analysis on the case study
4. Benchmark algorithms, objective functions, and hyperparameters

## Constraints

1. 1D simulations
2. Only scalar parameters (however, the framework allows more complex scenarios) 
3. No multi-objective (multiple targets must be squashed into a single one)

## Approach/Design Choices

* Python: simple language, good tooling e wide adoption in scientific computing community.
* Do not reinvent the wheel (use standard libraries as much as possible, otherwise third party libraries).
* Modularity, encapsulation and referential transparency.
* Emphasis on documentation and reproducibility.

## What has been done

<ol>
    <li class=fragment>Preliminary analysis of the case study and visualization</li>
    <li class=fragment>GEOtoPy: A wrapper based on IO and keywords</li>
    <li class=fragment>Prototype of calibration with Nevergrad and SA with SALib</li>
    <li class=fragment>Deployment on Ulysses, now moving to VSC-4</li>
</ol>

## GEOtop IO scheme

## GEOtoPy

![](data/assets/geotopy.svg)

In [3]:
class GEOtopExampleRun(gtp.GEOtop):

    def preprocess(self, working_dir, *args, **kwargs):

        settings = kwargs
        
        geotop_inpts_src = os.path.join(self.inputs_dir, 'geotop.inpts')
        geotop_inpts_dest = os.path.join(working_dir, 'geotop.inpts')
        os.remove(geotop_inpts_dest)
        
        with open(geotop_inpts_src, 'r') as src, open(geotop_inpts_dest, 'w') as geotop_inpts:
            geotop_inpts.write(f"! GEOtop input file written by GEOtoPy on {datetime.now()}\n")
            while line := src.readline():
                if gtp._comment_re.match(line):
                    geotop_inpts.write(line)
                else:
                    try:
                        key, value = gtp.read_setting(line)
                        if key in settings:
                            new_setting = gtp.print_setting(key, settings[key])
                            geotop_inpts.write(f"! {key} overwritten by GEOtoPy, was {value}\n")
                            geotop_inpts.write(new_setting)
                            del settings[key]
                        else:
                            geotop_inpts.write(gtp.print_setting(key, value))
                    except ValueError as err:
                        geotop_inpts.write(f"! ERROR: {err}")
                        geotop_inpts.write(line)
            
            if settings:
                geotop_inpts.write("\n! Settings added by GEOtoPy\n")
                for key, value in settings.items():
                    try:
                        geotop_inpts.write(gtp.print_setting(key, value))
                    except ValueError as err:
                        geotop_inpts.write(f"! ERROR: {err}")
                            
        
    def postprocess(self, working_dir):
        
        liq_path = os.path.join(working_dir, 'theta_liq.txt')
        liq = pd.read_csv(liq_path, 
                          na_values=['-9999'],
                          usecols=[0, 6, 7], 
                          skiprows=1,
                          header=0, 
                          names=['date', 'soil_moisture_content_50', 'soil_moisture_content_200'],
                          parse_dates=True, 
                          infer_datetime_format=True,
                          index_col=0)
        
        ice_path = os.path.join(working_dir, 'theta_ice.txt')
        ice = pd.read_csv(ice_path, 
                          na_values=['-9999'], 
                          usecols=[0, 6, 7], 
                          skiprows=1,
                          header=0, 
                          names=['date', 'soil_moisture_content_50', 'soil_moisture_content_200'],
                          parse_dates=True, 
                          infer_datetime_format=True,
                          index_col=0)
        
        sim = ice + liq
        
        return sim

In [4]:
class observations(Mapping):
    
    def __init__(self, path):
        
        self.data = pd.read_csv(path, 
                                na_values=['-9999'], 
                                parse_dates=True, 
                                infer_datetime_format=True,
                                index_col=0)
        
        self.data.index.rename('date', inplace=True)
        
        self.squared_mean = (self.data * self.data).mean()
        
    def __getitem__(self, key):
        
        return self.data[key]

    def __len__(self):
        
        return len(self.data)

    def __iter__(self):
        
        return iter(self.data)

    def compare(self, target, simulation, periods=None, name=None, unit=None, rel=False):

        if not periods:
            periods = {'Daily': 'D', 'Weekly': 'W', 'Monthly': 'M'}

        fig, axes = plt.subplots(ncols=3, 
                                 nrows=len(periods), 
                                 constrained_layout=True)

        if name:
            fig.suptitle(name)

        for i, (Tstr, T) in enumerate(periods.items()):
            comp_plot, diff_plot, hist_plot = axes[i, :]
            
            obs_resampled = self[target].resample(T).mean()
            sim_resampled = simulation[target].resample(T).mean()

            err = obs_resampled - sim_resampled        
            if rel:
                err = err / obs_resampled.abs()

            data = pd.DataFrame({'Observations': obs_resampled, 'Simulation': sim_resampled})
            sns.lineplot(data=data, ax=comp_plot)
            comp_plot.set_title(Tstr)
            comp_plot.set_xlabel("")
            if unit:
                comp_plot.set_ylabel(f'[{unit}]')

            sns.lineplot(data=err, ax=diff_plot)
            plt.setp(diff_plot.get_xticklabels(), rotation=20)
            if rel:
                diff_plot.set_ylabel(f'Relative error')
                diff_plot.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
            elif unit:
                diff_plot.set_ylabel(f'Error [{unit}]')
            else:
                diff_plot.set_ylabel(f'Error')

            sns.distplot(err, rug=True, vertical=True, hist=True, ax=hist_plot)
            y1, y2 = diff_plot.get_ylim()
            hist_plot.set_ylim(y1,y2)
            hist_plot.set_yticklabels([])
            hist_plot.set_ylabel("")
        
        return fig
    
    def metric(self, target, simulation):

        diff = (self[target] - simulation[target])
        diff_squared = diff * diff

        return np.sqrt(diff_squared.mean() / self.squared_mean[target])

In [5]:
model = GEOtopExampleRun('data/inputs/run',
                         exe='../../geotop/build/geotop',
                         run_args={'check': True, 
                                   'capture_output': True, 
                                   'timeout': timeout})

obs = observations('data/inputs/obs.csv')

INFO:numexpr.utils:NumExpr defaulting to 4 threads.


In [6]:
with tempfile.TemporaryDirectory() as tmpdir:
    sim = model.eval(tmpdir)
    print(f"Before optimization loss is {obs.metric('soil_moisture_content_50', sim)}")
    fig = obs.compare('soil_moisture_content_50', sim, name="Soil moisture content @ 50 mm", rel=True)
    fig.savefig('data/outputs/before.png')
    plt.close(fig)

Before optimization loss is 0.20770137763934485


![](data/outputs/before.png)

## Example of Sensitivity Analysis

In [7]:
def loss(*args, **kwargs): 
        
    with tempfile.TemporaryDirectory(dir=systmpfs) as tmpdir:
        try:
            sim = model.eval(tmpdir, *args, **kwargs)
        except subprocess.CalledProcessError:
            return np.nan
        except subprocess.TimeoutExpired:
            return np.nan

    return obs.metric('soil_moisture_content_50', sim)

In [8]:
def loss_SA_wrapper(kwargs):
    return loss(**kwargs)

problem = {'num_vars': 2,
           'names': ['ThetaRes', 'ThetaSat'],
           'bounds': [[0.0, 0.3], [0.4, 1.0]]}

samples = saltelli.sample(problem, 2)

settings = [{key: val for key, val in zip(problem['names'], values)} 
            for values in samples]

with Pool(processes=num_workers) as pool:
    losses = pool.map(loss_SA_wrapper, settings)

In [9]:
sobol.analyze(problem, np.array(losses), print_to_console=True);

Parameter S1 S1_conf ST ST_conf
ThetaRes -0.117488 0.446715 0.046895 0.008349
ThetaSat 2.286755 0.108141 2.404701 1.222927

Parameter_1 Parameter_2 S2 S2_conf
ThetaRes ThetaSat 0.390094 0.319298


## Example of Calibration

In [10]:
parameters = ng.p.Instrumentation(ThetaRes=ng.p.Scalar(lower=0.0, upper=0.3),
                                  ThetaSat=ng.p.Scalar(lower=0.3, upper=1.0))

optimizer = ng.optimizers.registry[algorithm](parametrization=parameters, 
                                              budget=budget, 
                                              num_workers=num_workers)

with ProcessPoolExecutor(max_workers=optimizer.num_workers) as executor:
    recommendation = optimizer.minimize(loss, 
                                        executor=executor, 
                                        batch_mode=False)

In [11]:
with tempfile.TemporaryDirectory() as tmpdir:
    print(f"Recommended value is: {recommendation.kwargs}")
    print(f"After optimization loss is: {recommendation.loss}")
    sim = model.eval(tmpdir, **recommendation.kwargs)
    fig = obs.compare('soil_moisture_content_50', sim, name=f"Soil moisture content @ 50mm", rel=True)
    fig.savefig('data/outputs/after.png')
    plt.close(fig)

Recommended value is: {'ThetaRes': 0.14515087014241174, 'ThetaSat': 0.43681851195135174}
After optimization loss is: 0.19726593860843625


![](data/outputs/after.png)

## Deployment

1. SPACK
2. PyPi
3. Pipenv
4. Papermill

## To Do

1. Assemble the calibration package
2. Benchmarks

## Further developments

1. Ray Tune + Nevergrad (multinode)?
2. Multinode SA with Ray?
3. Inoculate information from SA into optimizer?
4. IO optimizations?
5. BFGS after global optimization?
6. Parameters clustering?